# GradCAM Evaluation

### GradCAM Implementation

In [19]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        # Automatically find the last convolutional layer if none is provided
        if self.layerName is None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        # Find the last convolutional layer
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:  # Check for 4D outputs
                return layer.name
        raise ValueError("Could not find a 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        # Build a model that maps the inputs to the activations of the last conv layer and the outputs
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, self.model.output]
        )

        # Record operations for gradient computation
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        # Compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # Compute guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        # Average the gradients
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1, 2))
        
        # Compute the weighted sum of feature maps
        convOutputs = convOutputs[0]
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # Resize heatmap to match input image dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        # Normalize the heatmap
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_JET):
        # Apply colormap to heatmap
        heatmap = cv2.applyColorMap(heatmap, colormap)

        # Overlay heatmap on the image
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        return (heatmap, output)

def preprocess(img):
    # Resize to match the model's input dimensions
    img = cv2.resize(img, (224, 224))
    # Convert the image to RGB if needed (ensure 3 channels)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Normalize pixel values to [0, 1] or preprocess as per the model's requirements
    return img / 255.0  # Normalize to [0, 1]

# Example GradCAM Execution
test = cv2.imread("../images/20160928-140314-0.jpg")
test = cv2.resize(test, (224, 224))  # Ensure image is resized to model input dimensions
test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)

# Preprocess the test image
preprocessed_test = preprocess(test)
preprocessed_test = np.expand_dims(preprocessed_test, axis=0)

# Load your trained model (replace with your model)
model = tf.keras.applications.MobileNetV2(weights="imagenet")
model = Model(inputs=model.input, outputs=model.output)  # Ensure it's compatible with GradCAM

# Predict the class of the test image
predictions = model.predict(preprocessed_test)
predicted_class = np.argmax(predictions[0])

# Initialize GradCAM
cam = GradCAM(model, predicted_class)
heatmap = cam.compute_heatmap(preprocessed_test)

# Overlay heatmap on the original image
(heatmap, output) = cam.overlay_heatmap(heatmap, test, alpha=0.6)

# Display the results
plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.imshow(test / 255.0)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title("GradCAM Overlay")
plt.axis("off")

plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step


AttributeError: 'Dense' object has no attribute 'output_shape'